# Homework 2 Part 1

## Course Name: Large Language Models
#### Lecturers: Dr. Soleimani, Dr. Rohban, Dr. Asgari

---

#### Notebooks Supervised By: MohammadAli SadraeiJavaheri, Omid Ghahroodi
#### Notebook Prepared By: Omid Ghahroodi

**Contact**: Ask your questions in Quera

---

### Instructions:
- Complete all exercises presented in this notebook.
- Ensure you run each cell after you've entered your solution.
- After completing the exercises, save the notebook and <font color='red'>follow the submission guidelines provided in the PDF.</font>


---

**Note**: Replace the placeholders (between <font color="green">`## Your code begins ##`</font> and <font color="green">`## Your code ends ##`</font>) with the appropriate details.


## 1. Introduction

In this notebook, you will learn about data preprocessing for training a large language model and tokenizers.

In this notebook first, you download the raw data, then implement a pipeline for preprocessing this data. Finally, you train the three different types of tokenizers that you learned in the class on your data and compare them.

Answer the questions marked in green font.


In [ ]:
# Note: Do NOT make changes to this block.

!pip install transformers
!pip install fasttext
!pip install hazm


from IPython.display import clear_output
from huggingface_hub import hf_hub_download
from tqdm.notebook import tqdm
import numpy as np
import random
import transformers
import gzip
import json
import fasttext
from hazm import *

normalizer = Normalizer()

SEED=21

np.random.seed(SEED)
random.seed(SEED)

clear_output()

### Download Raw Data

In this section, download the raw data.

In [ ]:
# Note: Do NOT make changes to this block.

!GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/datasets/allenai/c4
%cd c4
!git lfs pull --include "c4-fa.tfrecord-00000-of-01024.json.gz"
%cd ..

file_path = '/content/c4/multilingual/c4-fa.tfrecord-00000-of-01024.json.gz'

with gzip.open(file_path, 'rb') as f:
    file_content = f.read().decode('utf8')

data = file_content.split('\n')

text_data = []

for i in data:
    if i!='':
        text_data.append(json.loads(i)['text'])

!wget https://raw.githubusercontent.com/roshan-research/hazm/master/hazm/data/stopwords.dat
with open('stopwords.dat', 'r') as f:
    stop_words = f.read()
stop_words = stop_words.rstrip()
stop_words = stop_words.split('\n')

clear_output()

### Statistics (5 points)

In this section, you should report several statistical characteristics of the data:
- Average length of the documents
- Largest length of the documents
- Smallest length of the documents
- The number of words in the data
- The most frequent word

In [ ]:
## Your code begins ##
from statistics import mean
from collections import Counter

# Calculate document lengths
doc_lengths = [len(doc) for doc in text_data]

avg_length = mean(doc_lengths)
print(f"Average Length of Documents: {avg_length:.2f}")

max_length = max(doc_lengths)
print(f"Largest Length of Documents: {max_length}")

min_length = min(doc_lengths)
print(f"Smallest Length of Documents: {min_length}")

# Tokenize documents and count words
all_words = [word for doc in text_data for word in word_tokenize(doc)]

total_words = len(all_words)
print(f"Number of Words in the Data: {total_words}")

unique_words = len(set(all_words))
print(f"Number of Unique Words in the Data: {unique_words}")

word_counts = Counter(all_words)
most_frequent_word, frequency = word_counts.most_common(1)[0]
print(f"Most Frequent Word: '{most_frequent_word}' (Frequency: {frequency} times)")

## Your code ends ##

Average Length of Documents: 2662.13
Largest Length of Documents: 168632
Smallest Length of Documents: 53
Number of Words in the Data: 29548463
Number of Unique Words in the Data: 791999
Most Frequent Word: 'و' (Frequency: 1033072 times)


In [ ]:
total_words

29548463

In [ ]:
# Note: Do NOT make changes to this block.
# Note: Do NOT change orginal_text_data

orginal_text_data = text_data[:]
len(orginal_text_data)


52663

### Low-quality Examples

In this section, several examples of poor-quality data from each category are provided. At the end of the preprocessing pipeline, you must remove such data from the original dataset to obtain high-quality data.

In [ ]:
# Note: Do NOT make changes to this block.

non_persian_example = orginal_text_data[9]
print(non_persian_example)


آذر ۷, ۱۳۹۷ - دیمه نیوز
Sportsbook Sites and Football Wagering Tips
Football gambling on tips-football playing advice on line The This country's Bookie Sportsbook, as a good useful support, delivers this kind of perceptive sections technique side bet your preferred sporting such as hockey, soccer, basketball game, tennis, sports, moose race, NASCAR, rugby, plus world of golf. Family home " Much more " Sports Bets ...
Major Business Strategies Selections Your solution to organization will most likely be influenced by the end objective. Then you can observe how without problems you operate your organization with low concerns in the foreseeable future. No matter of what topic you select to your new marketing and advertising business, ...
Top rated Business Strategies Options Your approach to organization is to damaged by the end target. Then you are going to observe how easily you run your company with very minimal issues down the road. Irrespective of what issue you select to your new on

In [ ]:
# Note: Do NOT make changes to this block.

short_length_example = orginal_text_data[1494]
print(short_length_example)


﻿ ﻿ مرجع آموزش زبان ایرانیان - _دانلود_نرم_افزار_سطح_مقدماتی_Oxford_Word_Skills_Basic_ ﻿


In [ ]:
# Note: Do NOT make changes to this block.

mean_length_word_example = orginal_text_data[250]
print(mean_length_word_example)


دو جمله حکیمانه : روانشناسی و اعتیاد - Page 9
159 پست • صفحه 9 از 11 • 1 ... 6, 7, 8, 9, 10, 11
[ 접속주소:opxx3.COM] &rdquo;&rdquo; 안동 섹시 &rdquo;&rdquo;
توسط 97WzugSKd3 » 6 مرداد 1397, 22:27
 [opxx3.COM]  &rdquo;☚&rdquo; 고양 강남 출장 만남 &there4;
توسط VTR8ve0Hhj » 6 مرداد 1397, 00:36
URL &rarr; [opxx3.COM] &rdquo;&rdquo; 김포 출장샵 &rdquo;✌&r
توسط PjTf4lb7wh » 6 مرداد 1397, 00:39
[opxx3.COM] &rdquo;&rdquo; 경주 출장안마 ※
توسط 97WzugSKd3 » 7 مرداد 1397, 06:02
[ 접속주소:opxx3.COM] &rdquo;&rdquo; 김천 서비스 &rdquo;&rdquo;
توسط rH7yB6bGKL » 7 مرداد 1397, 08:18
[opxx3.COM] &rdquo;&rdquo; 김천 콜걸 &rdquo;&rdquo;
توسط J5HDhyr3Hc » 7 مرداد 1397, 08:18
[opxx3.COM] &rdquo;&rdquo; 포항 강남 출장 아로마 &rdquo;&rdquo;
توسط J5HDhyr3Hc » 7 مرداد 1397, 08:20
WEBSITE  [opxx3.COM]  ※ 남양주 부평 출장 &rdquo;☚&rdquo;
توسط 97WzugSKd3 » 7 مرداد 1397, 12:23
[url=ï»¿https://stackoverflow.com/search?q=%F0%9F%8C%8F%20%5Bopxx3.COM%5D%20%F0%9F%8C%8F%2B%E2%80%9D%E2%9C%8C%E2%80%9D%2B%E2%80%9D%F0%9F%A4%A3%E2%80%9D%2B%E2%80%9D%F0%9F%8D%94%E2%8

In [ ]:
# Note: Do NOT make changes to this block.

symbol_ration_example = orginal_text_data[15018]
print(symbol_ration_example)


آموزش اضافه کردن تب های admincp و modcp به وی بی ایران خوش آمدید
ثبت نام کنید یا وارد شوید صفحه اصلی سایت مقالات رسانه تصویری آپلودسنتر فروشگاه دانلودر پست های جدید پرسش و پاسخ تقویم ابزار انجمن انتخاب بعنوان خوانده شده کلیدهای میانبر ارسال های امروز نمایش مدیران انجمن User Tagging Statistics Hash Tag Subscriptions وی بولتین مقالات آموزش اضافه کردن تب های admincp و modcp تاپیک به صورت خودکار بعد از 5 ثانیه آپدیت میشود بنابراین برای نمایش پست های جدید نیازی به رفرش صفحه نیست حالت ریفرش اتوماتیک به علت بی توجهی شما به این صفحه غیر فعال شد . «روشن کردن ریفرش اتوماتیک»
مقاله: آموزش اضافه کردن تب های admincp و modcp لینک بک آدرس لینک بک درباره لینک بک ها اضافه کردن به علاقه مندی / اشتراک فرستادن این موضوع به Digg !اضافه کردن موضوع به Delicious !اضافه کردن به علاقه مندی تکنوراتیفرستادن موضوع به Twitter ! ابزار مقاله پرینت این صفحه / حالت نمایش بصورت پرینت شده ارسال این صفحه به ایمیل یک دوست یا خودتان… Subscribe to this Article… نحوه نمایش موضوع حالت خطی تعویض به حالت ترکیبی تعوض به حالت رشته

In [ ]:
# Note: Do NOT make changes to this block.

alphabet_example = orginal_text_data[1]
print(alphabet_example)


املاک -- مشاهده تمام آگهی های دسته -- اجاره املاک اداری تجاری تهران ( 3) اجاره املاک مسکونی تهران ( 2) خرید و فروش آپارتمان ( 21) خرید و فروش املاک اداری تجاری تهران ( 3) خرید و فروش املاک مسکونی تهران ( 1) خرید و فروش خانه ( 3) خرید و فروش زمین ( 5) رهن و اجاره آپارتمان ( 5) رهن و اجاره خانه ( 1) سایر ( 9) کلنگی ( 2) مغازه و غرفه ( 3) ویلا ( 1) کالاو لوازم -- مشاهده تمام آگهی های دسته -- اثاثیه منزل ( 14) اداری ( 11) الکترونیک و دیجیتال ( 23) ایمنی ( 0) -- دوربین مدار بسته بازی و سرگرمی ( 1) برقی و گازی ( 12) بهداشتی ساختمان ( 9) بهداشتی و آرایشی ( 20) پزشکی ( 28) پوشاک ( 14) تزئینی , پرده , کرکره و ... ( 8) تزیینی ( 12) تعمیرگاهی ( 0) چوبی و فلزی ( 6) خیاطی و بافندگی ( 2) رستوران و فست فود ( 5) زیور آلات ( 1) ساعت ( 0) سایر ( 29) سرمایشی و گرمایشی ( 4) سیسمونی و نوزاد ( 0) شکار و ماهیگیری ( 1) صنعتی ( 5) صوتی و تصویری ( 9) عینک ( 0) فرش و زیر انداز ( 1) فیلم برداری و عکاسی ( 0) کالا و تجهیزات برقی ( 6) کتاب و مجلات ( 4) کیف و کفش ( 0) لوازم التحریر ( 1) لوازم ساختمانی ( 19) لوازم فرو

In [ ]:
# Note: Do NOT make changes to this block.

stopword_example = orginal_text_data[74]
print(stopword_example)

بقشاب پرنده های زیر آبی :: پنجره ی جهان نما
بقشاب پرنده های زیر آبی
چهارشنبه, ۱ مرداد ۱۳۹۳، ۰۷:۵۷ ب.ظ
منبع : مجله ی اینترنتی دانستنیها


## 2. Preprocessing (85 points)

In this part, you have to implement the pre-processing pipeline. This pipeline consists of the following sections:

1- Language Identification: In this section, you need to identify the language of the data using FastText and remove any data that is not in Farsi.

2- Length Filter: It is likely that examples that are shorter or longer than one length do not contain complete sentences or textual data. In this section, you should remove this data.

3- Mean Word Length: In high quality text, words should have an average number of characters within a certain range. you should find the average word length in each sample and exclude samples that fall outside the range.

4- Symbol Ratio: To ensure that a text is of good quality for training a model, the ratio of symbol count to word count should not exceed the threshold. In this task, you should calculate the ratio of symbol count to word count in any samples and filter out any samples with a ratio above a certain threshold.

5- Alphabetic Filter: In this part, you need to find the ratio of the number of words that contain at least one letter from the alphabet to the ratio of total number of words. If this ratio is below a certain limit, you should remove the sample.

6- Stop Word Filter: Find the number of stop words in a sample. If it's below a limit, delete the sample.

In [ ]:
tokenized_docs = [word_tokenize(text_row) for text_row in text_data]

In [ ]:
import numpy as np
import re

def calculate_percentiles(data, percent):
    """Calculate the percentiles of the data."""
    min_val = np.floor(np.percentile(data, percent))
    max_val = np.ceil(np.percentile(data, 100 - percent))
    return min_val, max_val

def calculate_ratios(tokens, condition):
    """Calculate the ratio of tokens that satisfy the condition."""
    return len(list(filter(condition, tokens))) / float(len(tokens))

def calculate_stopword_ratios(tokenized_docs, stop_words):
    """Calculate the ratio of stop words in each document."""
    return [len(list(filter(lambda token: token in stop_words, tokens))) for tokens in tokenized_docs]

def print_filters(filters):
    """Print the filters."""
    for name, value in filters.items():
        print(f"{name}: {value}")

# Define the percentile for length filtering
percent = 1

# Calculate the minimum and maximum lengths for length filtering
length_filter_min, length_filter_max = calculate_percentiles(doc_lengths, percent*5)

# Calculate the minimum and maximum lengths for word filtering
token_lengths = [[len(token) for token in tokens] for tokens in tokenized_docs]
word_lengths = np.array([[np.percentile(doc_token_lengths, 10*percent), np.percentile(doc_token_lengths, 100 - percent)] for doc_token_lengths in token_lengths])
word_length_filter_min, word_length_filter_max = np.percentile(word_lengths[::, 0], 100 - percent), np.percentile(word_lengths[::, 1], 100 - percent)
word_length_filter_min = np.floor(word_length_filter_min)
word_length_filter_max = np.ceil(word_length_filter_max)

# Define symbols
symbols = ["#", "*", "...", "@", "$", "&"]
symbols_regex = re.compile('|'.join(map(re.escape, symbols)))

# Calculate symbol_filter_max_ratio
symbol_count = [np.sum([float(len(symbols_regex.findall(word))) for word in tokens]) for tokens in tokenized_docs]
symbol_ratios = np.array(symbol_count, dtype=float)/doc_lengths

symbol_filter_max_ratio = np.percentile(symbol_ratios, 100 - percent*50)

# Calculate alphabetic_filter_min_ratio
alphabetic_filter_min_ratio = np.percentile([calculate_ratios(tokens, lambda token: token.isalpha()) for tokens in tokenized_docs], percent*20)

# Calculate stopword_ratios
stopword_ratios = calculate_stopword_ratios(tokenized_docs, stop_words)

# Calculate stop_words_filter_min_count
stop_words_filter_min_count = np.percentile(stopword_ratios, percent*20)
stop_words_filter_min_count = np.floor(stop_words_filter_min_count)

# Print the filters
filters = {
    "Length Filter Min": length_filter_min,
    "Length Filter Max": length_filter_max,
    "Word Length Filter Min": word_length_filter_min,
    "Word Length Filter Max": word_length_filter_max,
    "Symbol Filter Max Ratio": symbol_filter_max_ratio,
    "Stop Words Filter Min Count": stop_words_filter_min_count,
    "Alphabetic Filter Min Ratio": alphabetic_filter_min_ratio
}
print_filters(filters)


Length Filter Min: 116.0
Length Filter Max: 8611.0
Word Length Filter Min: 3.0
Word Length Filter Max: 20.0
Symbol Filter Max Ratio: 0.0
Stop Words Filter Min Count: 15.0
Alphabetic Filter Min Ratio: 0.7352941176470589


In [ ]:
def language_filter(text_data):
   model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
   model = fasttext.load_model(model_path)
   return_value = []
   for text_row in tqdm(text_data):
       text = text_row.replace("\n", " ")
       lang = model.predict(text)[0][0].replace('__label__', '')
       if lang in ['pes_Arab', 'prs_Arab']:
           return_value.append(text_row)
   return return_value

def length_filter(text_data, min_length=length_filter_min, max_length=length_filter_max):
   return_value = []
   for text_row in tqdm(text_data):
       length = len(word_tokenize(text_row))
       if length >= min_length and length <= max_length:
           return_value.append(text_row)
   return return_value

def word_length_filter(text_data, min_length=word_length_filter_min, max_length=word_length_filter_max):
   return_value = []
   for text_row in tqdm(text_data):
       words = word_tokenize(text_row)
       total_word_length = sum(len(word) for word in words)
       mean_word_length = total_word_length / len(words)
       if min_length <= mean_word_length <= max_length:
           return_value.append(text_row)
   return return_value

def symbol_filter(text_data, max_ratio=symbol_filter_max_ratio):
   symbols = ["#", "*", "...", "@", "$", "&"]
   return_value = []
   for text_row in tqdm(text_data):
       words = word_tokenize(text_row)
       total_symbols = sum(text_row.count(symbol) for symbol in symbols)
       total_words = len(words)
       symbol_to_word_ratio = total_symbols / total_words if total_words > 0 else 0
       if symbol_to_word_ratio <= max_ratio:
           return_value.append(text_row)
   return return_value

def alphabetic_filter(text_data, min_ratio=alphabetic_filter_min_ratio):
   return_value = []
   for text_row in tqdm(text_data):
       words = word_tokenize(text_row)
       alphabetic_word_count = sum(1 for word in words if word.isalpha())
       total_words = len(words)
       alphabetic_ratio = alphabetic_word_count / total_words if total_words > 0 else 0
       if alphabetic_ratio >= min_ratio:
           return_value.append(text_row)
   return return_value

def stop_words_filter(text_data, min_count=stop_words_filter_min_count):
   return_value = []
   for text_row in tqdm(text_data):
       words = word_tokenize(text_row)
       stop_word_count = sum(1 for word in words if word in stop_words)
       if stop_word_count >= min_count:
           return_value.append(text_row)
   return return_value

## Your code ends ##

### Sanity Check

In this cell, an simple check is done on the examples.

In [ ]:
# Note: Do NOT make changes to this block.

good_example = orginal_text_data[150]

assert (language_filter([non_persian_example, good_example])==[good_example])
assert (length_filter([short_length_example, good_example])==[good_example])
assert (word_length_filter([mean_length_word_example, good_example])==[good_example])
assert (symbol_filter([symbol_ration_example, good_example])==[good_example])
assert (alphabetic_filter([alphabet_example, good_example])==[good_example])
assert (stop_words_filter([stopword_example, good_example])==[good_example])


model.bin:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
word_length_filter([mean_length_word_example, good_example])

  0%|          | 0/2 [00:00<?, ?it/s]

['نجات ۱۰ کارگر گرفتار در تونل زره در چهار محال و بختیاری\nشهرکرد- مدیر عامل هلال احمر چهارمحال و بختیاری از نجات ۱۰ کارگر گرفتار در تونل زره در چهار محال و بختیاری خبر داد.\nسید احمد مرتضوی فرد در گفت و گو با خبرنگار مهر با اشاره به نجات ۱۰ کارگر گرفتار در تونل زره در چهار محال و بختیاری، اظهار داشت: این کارگران در این تونل مشغول فعالیت بودند که در پی بارش و ریزش در این تونل گرفتار شدند.\nوی بیان کرد: این ۱۰ کارگر با کمک نیرو\u200cهای امدادی هلال احمر نجات یافتند و به محل امن انتقال داده شدند.\nبارش باران از روز گذشته در استان چهارمحال و بختیاری آغاز شده است.']

### Normalization

In this cell, normalize the filtered texts.

Using `normalizer.normalize`.

In [ ]:
def normalize(text_data):
    return [
        ## Your code begins ##
        normalizer.normalize(text_row)
        ## Your code ends ##
        for text_row in tqdm(text_data)
    ]

### Pipeline

Run your preprocessing pipeline in this section.

In the input, give all the raw data, and in the output,  find high-quality data, and finally, normalize them.

Find the hyperparameters of the preprocessing pipeline so that at the end 60% of raw data remains. Hyperparameter list is:
- length_filter_min
- length_filter_max
- word_length_filter_min
- word_length_filter_max
- symbol_filter_max_ratio
- alphabetic_filter_min_ratio
- stop_words_filter_min_count

In [ ]:
total = len(text_data)

print(f'Total number of raw data: {total}')

text_data = language_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after language filtering')

text_data = length_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after length filtering')

text_data = word_length_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after mean word length filtering')

text_data = symbol_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after symbol ratio filtering')

text_data = alphabetic_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after alphabet filtering')

text_data = stop_words_filter(text_data)
print(f'{(len(text_data)/total)*100}% of raw data remained after stop word filtering')

text_data = normalize(text_data)


Total number of raw data: 52663


  0%|          | 0/52663 [00:00<?, ?it/s]

98.72775952756204% of raw data remained after language filtering


  0%|          | 0/51993 [00:00<?, ?it/s]

73.83931792719747% of raw data remained after length filtering


...

  0%|          | 0/25454 [00:00<?, ?it/s]

43.29415339042592% of raw data remained after alphabet filtering


  0%|          | 0/22800 [00:00<?, ?it/s]

42.80234699884169% of raw data remained after stop word filtering


  0%|          | 0/22541 [00:00<?, ?it/s]

### Save

In [ ]:
# Note: Do NOT make changes to this block.

with open('train.txt', 'w', encoding="utf-8") as f:
    f.write('\n'.join(text_data))

with open('raw_text.txt', 'w', encoding="utf-8") as f:
    f.write('\n'.join(orginal_text_data))

# 3. Tokenizer (10 points)

In this section, you will train tokenizers for cleaned and raw data. The training code is provided.

Finally, compare the output of the tokenizers.

### WordPiece Tokenizer

In [ ]:
# Note: Do NOT make changes to this block.

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

tokenizer_WordPiece = Tokenizer(models.WordPiece(unk_token="[UNK]"))

tokenizer_WordPiece.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.StripAccents()]
)

tokenizer_WordPiece.pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)

print(tokenizer_WordPiece.pre_tokenizer.pre_tokenize_str("این یک تست است!!!"))

special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer_WordPiece.train(["train.txt"], trainer=trainer)
tokenizer_WordPiece.save("tokenizer_WordPiece.json")

[('این', (0, 3)), ('یک', (4, 6)), ('تست', (7, 10)), ('است', (11, 14)), ('!', (14, 15)), ('!', (15, 16)), ('!', (16, 17))]


In [ ]:
# Note: Do NOT make changes to this block.

tokenizer_WordPiece2 = Tokenizer(models.WordPiece(unk_token="[UNK]"))

tokenizer_WordPiece2.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.StripAccents()]
)

tokenizer_WordPiece2.pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)


special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)
tokenizer_WordPiece2.train(["raw_text.txt"], trainer=trainer)
tokenizer_WordPiece2.save("tokenizer_WordPiece2.json")


In [ ]:
encoding = tokenizer_WordPiece.encode(text_data[0])
print(encoding.tokens)
encoding = tokenizer_WordPiece2.encode(text_data[0])
print(encoding.tokens)

['میکرو', '##کتاب', '۲۱', 'روز', 'تا', 'رسیدن', 'به', 'تحول', 'قدم\u200cهای', 'ساده', 'در', '۲۱', 'روز', 'برای', 'تقویت', 'روحیه', 'اضافه', 'کردن', 'نسخه', 'صوتی', 'فقط', 'با', '۲', ',', '۰۰۰', 'تومان', 'حتما', 'برای', 'شما', 'هم', 'پیش\u200cامده', 'که', 'از', 'شرایط', 'خود', 'راضی', 'نبود', '##ید', 'اما', 'هر', 'چه', 'تلاش', 'کردید', 'نتوانست', '##ید', 'اوضاع', 'را', 'بهبود', 'ببخشید', '.', 'لو', '##يی', '##ز', 'در', 'این', 'کتاب', 'می\u200cگوید', 'که', 'این', 'مشکل', 'از', 'درون', 'شماست', 'و', 'برای', 'رسیدن', 'به', 'انچه', 'می\u200cخواهید', 'باید', 'تمرین', 'کنید', '.', 'در', 'این', 'کتاب', '،', '۲۱', 'روز', 'برای', 'رسیدن', 'به', 'تحول', 'را', 'می\u200cگذ', '##رانید', '.', 'هر', 'فصل', 'مربوط', 'به', 'یک', 'روز', 'است', 'که', 'موضوعی', 'مختص', 'به', 'خودش', 'دارد', 'همینطور', 'هر', 'روز', 'تمرین', '##\u200cها', 'و', 'عبارات', 'تاکید', '##ی', 'دارد', 'که', 'بهتر', 'است', 'برای', 'انجام', 'انها', 'کاغذ', 'و', 'خودکار', 'همراه', '##تان', 'داشته', 'باشید', '.', 'لوی', '##یز', 'هی', '،

### BPE Tokenizer

In [ ]:
 # Note: Do NOT make changes to this block.

tokenizer_BPE = Tokenizer(models.BPE())
tokenizer_BPE.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
print(tokenizer_BPE.pre_tokenizer.pre_tokenize_str("این یک تست است!!!"))

trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer_BPE.train(["train.txt"], trainer=trainer)

tokenizer_BPE.save("tokenizer_BPE.json")


[('Ø§ÛĮÙĨ', (0, 3)), ('ĠÛĮÚ©', (3, 6)), ('ĠØªØ³Øª', (6, 10)), ('ĠØ§Ø³Øª', (10, 14)), ('!!!', (14, 17))]


In [ ]:
# Note: Do NOT make changes to this block.

tokenizer_BPE2 = Tokenizer(models.BPE())
tokenizer_BPE2.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

trainer = trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>"])
tokenizer_BPE2.train(["raw_text.txt"], trainer=trainer)

tokenizer_BPE2.save("tokenizer_BPE2.json")


In [ ]:
encoding = tokenizer_BPE.encode(text_data[0])
print(encoding.tokens)
encoding = tokenizer_BPE2.encode(text_data[0])
print(encoding.tokens)

['ÙħÛĮÚ©Ø±ÙĪ', 'Ú©ØªØ§Ø¨', 'ĠÛ²Û±', 'ĠØ±ÙĪØ²', 'ĠØªØ§', 'ĠØ±Ø³ÛĮØ¯ÙĨ', 'ĠØ¨Ùĩ', 'ĠØªØŃÙĪÙĦ', 'Ċ', 'ÙĤØ¯Ùħ', 'âĢĮ', 'ÙĩØ§ÛĮ', 'ĠØ³Ø§Ø¯Ùĩ', 'ĠØ¯Ø±', 'ĠÛ²Û±', 'ĠØ±ÙĪØ²', 'ĠØ¨Ø±Ø§ÛĮ', 'ĠØªÙĤÙĪÛĮØª', 'ĠØ±ÙĪØŃÛĮÙĩ', 'Ċ', 'Ø§Ø¶Ø§ÙģÙĩ', 'ĠÚ©Ø±Ø¯ÙĨ', 'ĠÙĨØ³Ø®Ùĩ', 'ĠØµÙĪØªÛĮ', 'ĠÙģÙĤØ·', 'ĠØ¨Ø§', 'ĠÛ²', 'Ġ,', 'ĠÛ°Û°Û°', 'ĠØªÙĪÙħØ§ÙĨ', 'Ċ', 'ØŃØªÙħØ§', 'ĠØ¨Ø±Ø§ÛĮ', 'ĠØ´ÙħØ§', 'ĠÙĩÙħ', 'ĠÙ¾ÛĮØ´', 'âĢĮ', 'Ø¢ÙħØ¯Ùĩ', 'ĠÚ©Ùĩ', 'ĠØ§Ø²', 'ĠØ´Ø±Ø§ÛĮØ·', 'ĠØ®ÙĪØ¯', 'ĠØ±Ø§Ø¶ÛĮ', 'ĠÙĨØ¨ÙĪØ¯', 'ÛĮØ¯', 'ĠØ§ÙħØ§', 'ĠÙĩØ±', 'ĠÚĨÙĩ', 'ĠØªÙĦØ§Ø´', 'ĠÚ©Ø±Ø¯ÛĮØ¯', 'ĠÙĨØªÙĪØ§ÙĨØ³Øª', 'ÛĮØ¯', 'ĠØ§ÙĪØ¶Ø§Ø¹', 'ĠØ±Ø§', 'ĠØ¨ÙĩØ¨ÙĪØ¯', 'ĠØ¨Ø¨Ø®Ø´ÛĮØ¯', '.', 'ĠÙĦÙĪ', 'Ø¦', 'ÛĮØ²', 'ĠØ¯Ø±', 'ĠØ§ÛĮÙĨ', 'ĠÚ©ØªØ§Ø¨', 'ĠÙħÛĮ', 'âĢĮ', 'Ú¯ÙĪÛĮØ¯', 'ĠÚ©Ùĩ', 'ĠØ§ÛĮÙĨ', 'ĠÙħØ´Ú©ÙĦ', 'ĠØ§Ø²', 'ĠØ¯Ø±ÙĪÙĨ', 'ĠØ´ÙħØ§Ø³Øª', 'ĠÙĪ', 'ĠØ¨Ø±Ø§ÛĮ', 'ĠØ±Ø³ÛĮØ¯ÙĨ', 'ĠØ¨Ùĩ', 'ĠØ¢ÙĨÚĨÙĩ', 'ĠÙħÛĮ', 'âĢĮ', 'Ø®ÙĪØ§ÙĩÛĮØ¯', 'ĠØ¨Ø§ÛĮØ¯', 'ĠØªÙħØ±ÛĮÙĨ', 'ĠÚ©ÙĨÛĮØ¯', '.', 'ĠØ¯Ø±', 'ĠØ§ÛĮÙĨ', 'ĠÚ©ØªØ§Ø¨', 'ØĮ', 'ĠÛ²Û±', 'ĠØ±ÙĪØ²', 'ĠØ¨Ø±Ø§ÛĮ', 'ĠØ±Ø³ÛĮØ

### Unigram Tokenizer

In [ ]:
# Note: Do NOT make changes to this block.

tokenizer_unigram = Tokenizer(models.Unigram())

tokenizer_unigram.normalizer = normalizers.Sequence(
    [
        normalizers.NFKD(),
        normalizers.StripAccents()
    ]
)

tokenizer_unigram.pre_tokenizer = pre_tokenizers.Metaspace()

tokenizer_unigram.pre_tokenizer.pre_tokenize_str("این یک تست است!!!")

special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=25000, special_tokens=special_tokens, unk_token="<unk>"
)
tokenizer_unigram.train(["train.txt"], trainer=trainer)


tokenizer_unigram.save("tokenizer_unigram.json")


In [ ]:
# Note: Do NOT make changes to this block.

tokenizer_unigram2 = Tokenizer(models.Unigram())

tokenizer_unigram2.normalizer = normalizers.Sequence(
    [
        normalizers.NFKD(),
        normalizers.StripAccents()
    ]
)

tokenizer_unigram2.pre_tokenizer = pre_tokenizers.Metaspace()

special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=25000, special_tokens=special_tokens, unk_token="<unk>"
)
tokenizer_unigram2.train(["raw_text.txt"], trainer=trainer)


tokenizer_unigram2.save("tokenizer_unigram2.json")


In [ ]:
encoding = tokenizer_unigram.encode(text_data[0])
print(encoding.tokens)
encoding = tokenizer_unigram2.encode(text_data[0])
print(encoding.tokens)

['▁میکرو', 'کتاب', '▁۲۱', '▁روز', '▁تا', '▁رسیدن', '▁به', '▁تحول', '\n', 'قدم', '\u200cهای', '▁ساده', '▁در', '▁۲۱', '▁روز', '▁برای', '▁تقویت', '▁روحیه', '\n', 'اضافه', '▁کردن', '▁نسخه', '▁صوتی', '▁فقط', '▁با', '▁۲', '▁,', '▁۰۰۰', '▁تومان', '\n', 'حت', 'ما', '▁برای', '▁شما', '▁هم', '▁پیش\u200cامده', '▁که', '▁از', '▁شرایط', '▁خود', '▁راضی', '▁نبود', 'ی', 'د', '▁اما', '▁هر', '▁چه', '▁تلاش', '▁کردید', '▁نتوانست', 'ی', 'د', '▁اوضاع', '▁را', '▁بهبود', '▁ببخشید', '.', '▁لويیز', '▁در', '▁این', '▁کتاب', '▁می\u200cگوید', '▁که', '▁این', '▁مشکل', '▁از', '▁درون', '▁شماست', '▁و', '▁برای', '▁رسیدن', '▁به', '▁انچه', '▁می\u200cخواهید', '▁باید', '▁تمرین', '▁کنید.', '▁در', '▁این', '▁کتاب', '،', '▁۲۱', '▁روز', '▁برای', '▁رسیدن', '▁به', '▁تحول', '▁را', '▁می\u200cگذرانید', '.', '▁هر', '▁فصل', '▁مربوط', '▁به', '▁یک', '▁روز', '▁است', '▁که', '▁موضوعی', '▁مختص', '▁به', '▁خودش', '▁دارد', '▁همینطور', '▁هر', '▁روز', '▁تمرین', '\u200cها', '▁و', '▁عبارات', '▁تاکید', 'ی', '▁دارد', '▁که', '▁بهتر', '▁است', '▁برای', '▁ا

### Analyze

In this part, WordPiece tokenizer and Unigram tokenizer are used on the data and the number of generated tokens and their intersection are displayed.

According to this section, answer the following questions.

<font color='green'>

1- Compare the output of tokenizers.

2- Why is the BPE output unreadable?

3- Analyze the intersection between WorkPiece tokens and Unigram tokens.

</font>

<font face='Roboto Arial' size='4'>

Answer:

1. The WordPiece and BPE tokenizers employ different subword tokenization methods, resulting in distinct outputs. While reviewing the output of the WordPiece tokenizer, it was observed that the tokenizer uses a double hash ## to denote the continuation of a word. For example, in the token list provided, '##کتاب' suggests the subword 'کتاب' is a continuation of the previous token 'میکرو'. This efficient division of words allows for better handling of rare words, maintaining semantics by retaining the association of subwords with their base words. <div/>
On the other hand, the BPE tokenizer output provided appears to use a byte-level representation, as indicated by tokens such as 'Ø§ÛĮÙĨ' and 'ĠÛĮÚ©'. This encoding captures the text on a much more granular level, which is beneficial for a language model that needs to handle a diverse set of characters, especially in the case of multilingual datasets. However, such byte-level representation is not directly human-readable and requires decoding into the respective characters for an understandable text output, which is evident from the comparison of the token outputs of the BPE tokenizer with those of the WordPiece tokenizer.

2. The BPE tokenizer outputs tokens that seem to be unreadably encoded. The reason for this unreadability is the use of byte-level operations that disassemble and reassemble words in a manner that does not correspond to human-readable characters directly. The output looks like a sequence of escape characters combined with Latin characters due to byte-level operations that represent non-ASCII characters or characters beyond the basic Latin alphabet. Unless these byte-representations are properly decoded into their corresponding characters, the resulting tokens will remain cryptic, as seen in the output excerpt provided.

3. WordPiece and Unigram are subword tokenization methods that handle out-of-vocabulary words by breaking them into smaller units. They start with a base vocabulary of individual characters and learn a richer vocabulary of common subwords and words in a dataset. WordPiece learns common subwords by iteratively finding the most frequent pairs of characters or character sequences, while Unigram starts with a large vocabulary and trims it down by calculating the likelihood of each subword combination and removing the least frequent ones.<div/>
Both tokenizers generate a substantial number of tokens from the input text, with Unigram producing slightly more unique tokens than WordPiece. This could be due to their different strategies in handling rare words and the specific characteristics of the input text. The intersection of tokens generated by the two tokenizers is much smaller than the total number of unique tokens produced by each, indicating that they have different ways of breaking down words into subword units. This difference might be beneficial in different contexts depending on the specific needs of the downstream tasks.<div/>
In conclusion, while both WordPiece and Unigram effectively break down words into meaningful subword units, they have different strengths and can be chosen based on the specific needs of the downstream tasks. The intersection of tokens they generate provides a common ground of subword units that are recognized as meaningful by both methods.

</font>

In [ ]:
unigram_decoding = []
wordpiece_decoding = []

for i in tqdm(text_data):
    wordpiece_decoding.append(tokenizer_WordPiece.encode(i).tokens)
    unigram_decoding.append(tokenizer_unigram.encode(i).tokens)


In [ ]:
import itertools

print(f'Number of tokens generated by unigram: {len(list(itertools.chain(*unigram_decoding)))}')
print(f'Number of tokens generated by wordpiece: {len(list(itertools.chain(*wordpiece_decoding)))}')

print(f'Number of unique tokens generated by unigram: {len(set(itertools.chain(*unigram_decoding)))}')
print(f'Number of unique tokens generated by wordpiece: {len(set(itertools.chain(*wordpiece_decoding)))}')

print(f'Number of intersection tokens: {len(list(set(itertools.chain(*unigram_decoding)) & set(itertools.chain(*wordpiece_decoding))))}')


Number of tokens generated by unigram: 14106739
Number of tokens generated by wordpiece: 13779688
Number of unique tokens generated by unigram: 24989
Number of unique tokens generated by wordpiece: 24346
Number of intersection tokens: 2955
